A hotel has accidentally made overbookings for certain rooms on specific dates. Due to this error, some rooms have been assigned to multiple customers for overlapping periods, leading to potential conflicts. The hotel management needs to rectify this mistake by contacting the affected customers and providing them with alternative arrangements.

 

Your task is to write an SQL query to identify the overlapping bookings for each room and determine the list of customers affected by these overlaps. For each room and overlapping date, the query should list the customers who have booked the room for that date. 
 

A booking's check-out date is not inclusive, meaning that if a room is booked from April 1st to April 4th, it is considered occupied from April 1st to April 3rd , another customer can check-in on April 4th and that will not be considered as overlap.
 

Order the result by room id, booking date. You may use calendar dim table which has all the dates for the year April 2024.

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import *
from pyspark.sql import Row
from datetime import datetime, timedelta

# Create SparkSession
spark = SparkSession.builder.appName("CreateBookingsTable").getOrCreate()

# Define the schema
schema = StructType([
    StructField("room_id", IntegerType(), True),
    StructField("customer_id", IntegerType(), True),
    StructField("check_in_date", StringType(), True),
    StructField("check_out_date", StringType(), True)
])

# Create the data
data = [
    (1, 101, "2024-04-01", "2024-04-04"),
    (2, 102, "2024-04-02", "2024-04-05"),
    (1, 103, "2024-04-02", "2024-04-06"),
    (3, 104, "2024-04-03", "2024-04-05"),
    (2, 105, "2024-04-04", "2024-04-07"),
    (1, 106, "2024-04-05", "2024-04-08"),
    (3, 107, "2024-04-05", "2024-04-09")
]

# Create the DataFrame
bookings = spark.createDataFrame(data, schema=schema)

# Show the DataFrame
bookings.show()


+-------+-----------+-------------+--------------+
|room_id|customer_id|check_in_date|check_out_date|
+-------+-----------+-------------+--------------+
|      1|        101|   2024-04-01|    2024-04-04|
|      2|        102|   2024-04-02|    2024-04-05|
|      1|        103|   2024-04-02|    2024-04-06|
|      3|        104|   2024-04-03|    2024-04-05|
|      2|        105|   2024-04-04|    2024-04-07|
|      1|        106|   2024-04-05|    2024-04-08|
|      3|        107|   2024-04-05|    2024-04-09|
+-------+-----------+-------------+--------------+



In [0]:
#Generating the calender table

start_date=datetime(2024,4,1)
end_date=datetime(2024,4,30)

date_list=[(start_date+timedelta(days=i)).strftime("%Y-%m-%d") for i in range((end_date-start_date).days+1)]

data=[(date,) for date in date_list]
schema_cal=StructType([
    StructField("cal_date",StringType(),True)
    ])
calendar_dim=spark.createDataFrame(data,schema=schema_cal)



In [0]:
from pyspark.sql.functions import col, collect_list, concat_ws, count

# Step 1: Create the CTE equivalent (joining bookings and calendar_dim)
cte = bookings.join(calendar_dim, 
                    (calendar_dim.cal_date >= bookings.check_in_date) & 
                    (calendar_dim.cal_date < bookings.check_out_date)) \
              .select(col("room_id"), col("customer_id"), col("cal_date").alias("book_date"))

# Step 2: Group by room_id and book_date, aggregate customer IDs
result_df = cte.groupBy(col("room_id"), col("book_date")) \
    .agg(
        concat_ws(",", collect_list(col("customer_id"))).alias("customers"),  # Comma-separated customer IDs
        count("*").alias("counting")  # Count the number of rows in each group
    ) \
        .filter(col("counting") > 1) \
            .orderBy(col("room_id"), col("book_date"))  # Order by room_id and book_date

# Step 3: Show the result
result_df.show()


+-------+----------+---------+--------+
|room_id| book_date|customers|counting|
+-------+----------+---------+--------+
|      1|2024-04-02|  101,103|       2|
|      1|2024-04-03|  101,103|       2|
|      1|2024-04-05|  103,106|       2|
|      2|2024-04-04|  102,105|       2|
+-------+----------+---------+--------+

